In [8]:
# -------------------------------
# Libraries
# -------------------------------
%matplotlib inline
import os.path
from math import *
import numpy as np
from tramway.helper import *
from tramway.helper.simulation import *
from tramway.helper.tessellation import *
from tramway.helper.inference import *
from tramway.plot.mesh import plot_delaunay
from numpy.random import seed
#from scipy.stats import skellam
#from scipy.optimize import fmin,minimize

#--------------------------------
# Functions
#--------------------------------
def D_field(r,t):
    '''Defining the diffusivity field: Gaussian in space, constant in time.'''
    return D - (D-D0)*np.exp(-np.sum((r/r_scale-mu_r)**2)/sig_r) 

def new_tracers(t):
    '''Number of tracers to appear in each frame n ~ Poisson(mu_on).'''
    return np.random.poisson(mu_on)

#--------------------------------
# Constants and parameters:
#--------------------------------
# Tesselation method and name:
tessellation_method = 'hexagon'
mesh_label = '{}'.format(tessellation_method) #, location_count, min_location_count)
min_loc_count = 0
# Optimization parameters:
nt_method = ('nT_test02_hex', 'NM')
tol = 1e-2

# Normalized placement and with of diffusivity well:
mu_r=np.array([0.5,0.5]) 
sig_r=0.05
# Size of the bounding box:
r_scale = 10. # [um]
# Reference distance to set mesh size:
ref_distance = 1. # [mu]

# baseline diffusivity:
D = 0.5 # [um^2.s^-1]
# diffusivity at the bottom of the sink:
D0 = 0.05 # [um^2.s^-1]
# number of trajectories:
M = 100
# time step:
dt = 0.05 # [s]
# average trajectory lifetime (+ p_off and mu_on):
tau = 0.25
p_off = dt/tau
mu_on = p_off*M
# Simulation duration:
T = 0.25 # [s]
# position noise:
sigma = 0. # [um]
# Number of ensembles to test:
Q = 1


In [ ]:
#--------------------------------
# Simulate:
#--------------------------------
seed(1234567890)
## generate the trajectories
q=0

nxyt = random_walk(diffusivity=D_field, duration=T, lifetime_tau=tau, single=True, reflect=True, 
                  trajectory_mean_count=M, box=[0.,0.,r_scale,r_scale], )
nxyt = nxyt.dropna()
xyt = nxyt.drop('n',axis=1)

# tesselate for trajectories:
cells_nxyt = tessellate(nxyt, tessellation_method, ref_distance=ref_distance, min_location_count=0,
           strict_min_location_count=min_loc_count,
           force=True, output_label=mesh_label)
# tesselate for positions:
cells_xyt = tessellate(xyt, tessellation_method, ref_distance=ref_distance, min_location_count=0,
           strict_min_location_count=min_loc_count,
           force=True, output_label=mesh_label)


## plot for trajectories and for positions:
cell_plot(cells_nxyt, cell_indices=True)
cell_plot(cells_xyt, cell_indices=True)
map_plot((pd.DataFrame(cells_xyt.location_count, columns=['N'])),cells=cells_xyt) #,cmap='inferno') 

In [11]:
# ------------------------------------------------
# infer in D mode without spatial regularization
# ------------------------------------------------
maps_nxyt = infer(cells_nxyt, 'D', 
             sigma=sigma, verbose=False)

print(maps_nxyt)

mode:      d
runtime:   0.02901935577392578
sigma:     0.0
variables: ['diffusivity']
maps:      <class 'pandas.core.frame.DataFrame'>


In [ ]:
# ------------------------------------------------
# infer using nontracking (D mode without spatial regularization only)
# ------------------------------------------------
maps_xyt = infer(cells_xyt, nt_method[0], new_cell=Locations, dt=dt, p_off=p_off, mu_on=mu_on,
        method=nt_method[1], tol=tol)



S: 0 3.4641016151377544
1 3.4641016151377544
6 6.928203230275509
; total: 10.392304845413264 mu^2
Cell no.: 0 ; sum(N): 1 , possible links: 0 , avg(N): 0.2
Outer cells: sum(N): 15 , possible links: 10 , avg(N): 3.0
All cells: sum(N): 16 , possible links: 11 , avg(N): 3.2
[1, 0, 0, 0, 0]
[1, 2, 4, 5, 4]
 final_simplex: (array([[0.345 , 0.1275],
       [0.34  , 0.135 ],
       [0.34  , 0.125 ]]), array([8.76544076, 8.76551681, 8.76551681]))
           fun: 8.765440761158914
       message: 'Optimization terminated successfully.'
          nfev: 22
           nit: 13
        status: 0
       success: True
             x: array([0.345 , 0.1275])
D corrected for motion blur and localization error: 0.4425000000000008
0.029000520706176758 s

S: 1 3.4641016151377544
0 3.4641016151377544
2 6.928203230275509
6 10.392304845413264
7 13.856406460551018
; total: 17.32050807568877 mu^2
Cell no.: 1 ; sum(N): 4 , possible links: 2 , avg(N): 0.8
Outer cells: sum(N): 33 , possible links: 24 , avg(N): 6.

In [ ]:
centers = cells_nxyt.tessellation.cell_centers
D_true = pd.DataFrame([D_field(c_,0.) for c_ in centers],columns=['D'])
maps_true = Maps(D_true)
map_plot(maps_true, cells=cells_nxyt, title=False, cmap='inferno', clim=[0,1])

## plot the force
map_plot(maps_nxyt, cells=cells_nxyt, title=False, variable='diffusivity',cmap='inferno',clim=[0,1])
# other variables are 'diffusivity' and 'potential'

map_plot(maps_xyt, cells=cells_xyt, title=False, cmap='inferno',clim=[0,1])

maps_diffs = Maps(pd.DataFrame(maps_xyt.maps['D'] - maps_nxyt.maps['diffusivity'],columns=['D']))
map_plot(maps_diffs, cells=cells_xyt, title=False, cmap='inferno',clim=[0,1])


In [ ]:
analysis_tree_nxyt = Analyses(nxyt)
analysis_tree_nxyt.add(cells_nxyt, label='mesh')
analysis_tree_nxyt['mesh'].add(maps_nxyt, label='diffusivity')
print(analysis_tree_nxyt['mesh']['diffusivity'].data.maps)
save_rwa(f'analyses_nxyt,D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance}},q={q}/{Q}.rwa', analysis_tree_nxyt)

#maps_xyt = pd.read_csv('Estimates-1.txt', sep='\t', index_col=0, names=['diffusivity'])
analysis_tree_xyt = Analyses(xyt)
analysis_tree_xyt.add(cells_xyt, label='mesh')
analysis_tree_xyt['mesh'].add(maps_xyt, label='diffusivity')
print(analysis_tree_xyt['mesh']['diffusivity'].data.maps)
save_rwa(f'analyses_xyt,D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance},q={q}/{Q}.rwa', analysis_tree_xyt)


In [ ]:
maps_nxyt.data['diffusivity']